#### DATA WRANGLING

In [1]:
import pandas as pd
import numpy as np


In [2]:
df=pd.read_csv("train_main_loan.csv")
pd.set_option('display.max_columns', None)
df.head(5)

,ID,ACCOUNT_TYPE,HIGH_CREDIT_OR_SANCTIONED_AMOUNT,DATE_OPENED,CURRENT_BALANCE,ACTUAL_PAYMT_AMT,EMI_AMOUNT,REPAYMENT_TENURE,LOAN_CLASSIFICATION,AMOUNT_OVERDUE,PAYMENT_HISTORY_1,PAYMENT_HISTORY_2,OWNERSHIP_TYPE,COLLATERALVALUE,TU_SCORE,PAYMENT_HISTORY_START_DATE,PAYMENT_HISTORY_END_DATE,DATE_REPORTED_AND_CERTIFIED,DATE_OF_LAST_PAYMENT,Reported_Date,DATE_OF_BIRTH,OCCUPATION_TYPE,GENDER,ACTUAL_ROI
0,A002338349,Housing Loan,818517,13-06-2014,640742,8627.0,8627.0,142.0,0,NaN,0000000000000000000000000000000000000520520520...,3.00E+19,Joint,2086000.0,681,01-11-2023,01-12-2020,30-11-2023,30-11-2023,30-11-2023,21-05-1971,SALARIED,Male,12.32
1,A002000537,Housing Loan,1243755,05-12-2012,949446,500.0,12210.0,156.0,0,NaN,0000000000000000000000000000000000000000000000...,0,Joint,1536000.0,784,01-11-2023,01-12-2020,30-11-2023,30-11-2023,30-11-2023,12-11-1984,SALARIED,Female,12.42
2,A002421579,Housing Loan,1826422,16-03-2017,1296849,16036.0,16036.0,148.0,0,NaN,0000000000000000000000000000000000000000000000...,0,Joint,3587700.0,748,01-11-2023,01-12-2020,30-11-2023,11-11-2023,30-11-2023,01-08-1986,SALARIED,Male,10.87
3,A002152345,Housing Loan,1847916,24-12-2022,1724256,26996.0,NaN,NaN,0,NaN,0,NaN,Joint,3049600.0,786,01-11-2023,01-12-2022,30-11-2023,05-11-2023,30-11-2023,22-07-1969,NaN,Female,10.00
4,A001952834,Housing Loan,2318386,21-10-2014,1790937,25131.0,NaN,116.0,539,431551.0,5395705705405095065075075075075095095094794784...,4.50E+53,Joint,2785400.0,664,01-11-2023,01-12-2020,30-11-2023,27-11-2023,30-11-2023,28-10-1974,SALARIED,Male,11.97


#### *PAYMENT HISTORY 1 (MAKE IT MORE INTERPRATABLE BY THE MODEL)*

In [3]:
def preprocess_payment_history(payment_history):
    
    if payment_history == '0':
        return '0' * 54
    elif len(payment_history) == 54:
        return payment_history
    elif len(payment_history) < 54 and 'XXX' in payment_history:
        return payment_history + '0' * (54 - len(payment_history))
    elif 'E' in payment_history:
        # Convert to a standard number and then format it to a string
        calculated_value = format(float(payment_history), '.0f')
        return '0' * (54 - len(calculated_value)) + calculated_value
    elif len(payment_history) > 1 and 'XXX' not in payment_history:
        return '0' * (54 - len(payment_history)) + payment_history
    else:
        return '0'*54

# Apply the preprocessing function to the 'PAYMENT_HISTORY_1' column
df['PAYMENT_HISTORY_1'] = df['PAYMENT_HISTORY_1'].apply(preprocess_payment_history)


In [4]:
import pandas as pd


def wrangle_payment_history(payment_history):
    # Break the 54-character string into 3-character segments
    segments = [payment_history[i:i+3] for i in range(0, len(payment_history), 3)]

    # Feature 1: Number of times paid ('000')
    count_paid = segments.count('000')

    # Feature 2: Number of times late (non-zero but not 'XXX')
    count_late = sum(1 for segment in segments if segment != '000' and 'XXX' not in segment)

    # Feature 3: Number of times took more than 90 days
    count_more_than_90 = sum(1 for segment in segments if segment.isdigit() and int(segment) > 90)

    # Feature 4: Number of times took days between 0-60
    count_0_to_60 = sum(1 for segment in segments if segment.isdigit() and 0 < int(segment) <= 60)

    # Feature 5: Average days (non-'XXX' values, excluding '000' and 'XXX')
    non_xxx_values = [int(segment) for segment in segments if segment.isdigit() and segment != '000']
    avg_days = sum(non_xxx_values) / len(non_xxx_values) if non_xxx_values else 0

    # Feature 6: Number of missing days
    count_missing_days = sum(1 for segment in segments if 'XXX' in segment)

    return count_paid, count_late, count_more_than_90, count_0_to_60, avg_days, count_missing_days

# Apply the wrangling function to the 'PAYMENT_HISTORY_1' column and store results in separate columns
df[['COUNT_PAID', 'COUNT_LATE', 'COUNT_MORE_THAN_90', 'COUNT_0_TO_60', 'AVG_DAYS', 'COUNT_MISSING_DAYS']] = df['PAYMENT_HISTORY_1'].apply(wrangle_payment_history).tolist()



#### EXTRACTING NUMERICAL FEATURES FROM THE DATES

In [5]:
def date_opened_to_month(df):
  df['DATE_OPENED'] = pd.to_datetime(df['DATE_OPENED'], format='%d-%m-%Y')
  
  # Define the constant 'Reported_Date'
  constant_reported_date = pd.to_datetime('30-11-2023', format='%d-%m-%Y')

  # Calculate the number of months
  df['Months_Since_Opened'] = (constant_reported_date - df['DATE_OPENED']).dt.days // 30

  # Replace 'DATE_OPENED' with the calculated number of months
  df['DATE_OPENED'] = df['Months_Since_Opened']

  # Drop the 'Months_Since_Opened' column if you don't need it anymore
  df.drop(columns=['Months_Since_Opened'], inplace=True)
  df.rename(columns={'DATE_OPENED': 'MONTH_OPENED'}, inplace=True)

  return df

In [6]:
def dob_to_age(df):
  df['DATE_OF_BIRTH'] = pd.to_datetime(df['DATE_OF_BIRTH'], format='%d-%m-%Y')
  # Define the constant 'Reported_Date'
  constant_reported_date = pd.to_datetime('30-11-2023', format='%d-%m-%Y')
  # Calculate the number of years
  df['Years_Since_Birth'] = (constant_reported_date - df['DATE_OF_BIRTH']).dt.days // 365
  # Replace 'DATE_OF_BIRTH' with the calculated number of years
  df['DATE_OF_BIRTH'] = df['Years_Since_Birth']
  # Drop the 'Years_Since_Birth' column if you don't need it anymore
  df.drop(columns=['Years_Since_Birth'], inplace=True)
  df.rename(columns={'DATE_OF_BIRTH': 'AGE'}, inplace=True)
  return df

In [7]:
def date_certified_to_month(df):
  df['DATE_REPORTED_AND_CERTIFIED'] = pd.to_datetime(df['DATE_REPORTED_AND_CERTIFIED'], format='%d-%m-%Y')

  # Define the constant 'Reported_Date'
  constant_reported_date = pd.to_datetime('30-11-2023', format='%d-%m-%Y')

  # Calculate the number of months
  df['Months_Since_Opened'] = (constant_reported_date - df['DATE_REPORTED_AND_CERTIFIED']).dt.days // 30

  # Replace 'DATE_OPENED' with the calculated number of months
  df['DATE_REPORTED_AND_CERTIFIED'] = df['Months_Since_Opened']

  # Drop the 'Months_Since_Opened' column if you don't need it anymore
  df.drop(columns=['Months_Since_Opened'], inplace=True)
  df.rename(columns={'DATE_REPORTED_AND_CERTIFIED': 'MONTH_CERTIFIED'}, inplace=True)

  return df

In [8]:
def last_payment_to_month(df):
  df['DATE_OF_LAST_PAYMENT'] = pd.to_datetime(df['DATE_OF_LAST_PAYMENT'], format='%d-%m-%Y')

  # Define the constant 'Reported_Date'
  constant_reported_date = pd.to_datetime('30-11-2023', format='%d-%m-%Y')

  # Calculate the number of months
  df['Months_Since_Opened'] = (constant_reported_date - df['DATE_OF_LAST_PAYMENT']).dt.days // 30

  # Replace 'DATE_OPENED' with the calculated number of months
  df['DATE_OF_LAST_PAYMENT'] = df['Months_Since_Opened']

  # Drop the 'Months_Since_Opened' column if you don't need it anymore
  df.drop(columns=['Months_Since_Opened'], inplace=True)
  df.rename(columns={'DATE_OF_LAST_PAYMENT': 'MONTH_LAST_PAYMENT'}, inplace=True)

  return df

In [9]:
def history_start_end_dates(df):
  df['DATE_OF_LAST_PAYMENT'] = pd.to_datetime(df['DATE_OF_LAST_PAYMENT'], format='%d-%m-%Y')

  # Define the constant 'Reported_Date'
  constant_reported_date = pd.to_datetime('30-11-2023', format='%d-%m-%Y')

  # Calculate the number of months
  df['Months_Since_Opened'] = (constant_reported_date - df['DATE_OF_LAST_PAYMENT']).dt.days // 30

  # Replace 'DATE_OPENED' with the calculated number of months
  df['DATE_OF_LAST_PAYMENT'] = df['Months_Since_Opened']

  # Drop the 'Months_Since_Opened' column if you don't need it anymore
  df.drop(columns=['Months_Since_Opened'], inplace=True)
  df.rename(columns={'DATE_OF_LAST_PAYMENT': 'MONTH_LAST_PAYMENT'}, inplace=True)

  return df

In [10]:
def history_start_end_dates(df):
    # Convert 'PAYMENT_HISTORY_START_DATE' and 'PAYMENT_HISTORY_END_DATE' to datetime
    df['PAYMENT_HISTORY_START_DATE'] = pd.to_datetime(df['PAYMENT_HISTORY_START_DATE'], format='%d-%m-%Y')
    df['PAYMENT_HISTORY_END_DATE'] = pd.to_datetime(df['PAYMENT_HISTORY_END_DATE'], format='%d-%m-%Y')

    # Add your desired operations for 'PAYMENT_HISTORY_START_DATE' here (if needed)

    # Example operation: Calculate the number of months between 'PAYMENT_HISTORY_START_DATE' and a constant date
    constant_reported_date = pd.to_datetime('30-11-2023', format='%d-%m-%Y')
    df['Months_Since_Payment_Start'] = (constant_reported_date - df['PAYMENT_HISTORY_START_DATE']).dt.days // 30

    # Replace 'PAYMENT_HISTORY_START_DATE' with the calculated number of months
    df['PAYMENT_HISTORY_START_DATE'] = df['Months_Since_Payment_Start']

    df.drop(columns=['Months_Since_Payment_Start'], inplace=True)

    df.rename(columns={'PAYMENT_HISTORY_START_DATE': 'MONTH_PAYMENT_HISTORY_START'}, inplace=True)

    
    df['Months_Since_Payment_End'] = (constant_reported_date - df['PAYMENT_HISTORY_END_DATE']).dt.days // 30

    df['PAYMENT_HISTORY_END_DATE'] = df['Months_Since_Payment_End']

    # Drop the 'Months_Since_Payment_End' column if you don't need it anymore
    df.drop(columns=['Months_Since_Payment_End'], inplace=True)

    # Rename 'PAYMENT_HISTORY_END_DATE' to 'MONTH_PAYMENT_HISTORY_END'
    df.rename(columns={'PAYMENT_HISTORY_END_DATE': 'MONTH_PAYMENT_HISTORY_END'}, inplace=True)

    return df

### *applying these functions*

In [11]:
df_new=date_certified_to_month(df)
df_new=last_payment_to_month(df_new)
df_new=dob_to_age(df_new)
df_new=date_opened_to_month(df_new)
df_new=history_start_end_dates(df_new)

In [12]:
df_new.head(5)

,ID,ACCOUNT_TYPE,HIGH_CREDIT_OR_SANCTIONED_AMOUNT,MONTH_OPENED,CURRENT_BALANCE,ACTUAL_PAYMT_AMT,EMI_AMOUNT,REPAYMENT_TENURE,LOAN_CLASSIFICATION,AMOUNT_OVERDUE,PAYMENT_HISTORY_1,PAYMENT_HISTORY_2,OWNERSHIP_TYPE,COLLATERALVALUE,TU_SCORE,MONTH_PAYMENT_HISTORY_START,MONTH_PAYMENT_HISTORY_END,MONTH_CERTIFIED,MONTH_LAST_PAYMENT,Reported_Date,AGE,OCCUPATION_TYPE,GENDER,COUNT_PAID,COUNT_LATE,COUNT_MORE_THAN_90,COUNT_0_TO_60,AVG_DAYS,COUNT_MISSING_DAYS
0,A000614177,Housing Loan,2277986,91,1824507,22985.0,NaN,149.0,0,NaN,0000000000000000000000000000000000000000000000...,0,Joint,3780000.0,780,0,36,0,0.0,30-11-2023,35,SEP,Male,17.0,0.0,0.0,0.0,0.0,1.0
1,A001744326,Business Loan,1030000,14,819209,30256.0,NaN,NaN,0,NaN,0000000000000000000000000000000000000000000000...,NaN,Joint,NaN,774,0,15,0,0.0,30-11-2023,47,NaN,Male,18.0,0.0,0.0,0.0,0.0,0.0
2,A000242802,Housing Loan,960705,3,908604,25229.0,NaN,NaN,0,NaN,0000000000000000000000000000000000000000000000...,NaN,Joint,2622010.0,809,0,4,0,0.0,30-11-2023,59,SENP,Female,18.0,0.0,0.0,0.0,0.0,0.0
3,B1000244547,Housing Loan,1437990,69,1367609,13067.0,NaN,259.0,0,NaN,0000000000000000000000000000000000000000000000...,0,Joint,2947100.0,759,0,36,0,0.0,30-11-2023,54,SENP,Male,17.0,0.0,0.0,0.0,0.0,1.0
4,B3000178046,Business Loan,728474,1,718224,22350.0,21778.0,NaN,0,NaN,0000000000000000000000000000000000000000000000...,NaN,Joint,NaN,731,0,2,0,0.0,30-11-2023,50,SENP,Male,18.0,0.0,0.0,0.0,0.0,0.0


### *AT LAST THERE ARE SOME COLUMNS TO DROP THAT ARE NOT SIGNIFICANT*

In [13]:
columns_to_drop = ['Reported_Date', 'PAYMENT_HISTORY_2']
df_new = df_new.drop(columns=columns_to_drop)

In [14]:
df_new

,ID,ACCOUNT_TYPE,HIGH_CREDIT_OR_SANCTIONED_AMOUNT,MONTH_OPENED,CURRENT_BALANCE,ACTUAL_PAYMT_AMT,EMI_AMOUNT,REPAYMENT_TENURE,LOAN_CLASSIFICATION,AMOUNT_OVERDUE,PAYMENT_HISTORY_1,OWNERSHIP_TYPE,COLLATERALVALUE,TU_SCORE,MONTH_PAYMENT_HISTORY_START,MONTH_PAYMENT_HISTORY_END,MONTH_CERTIFIED,MONTH_LAST_PAYMENT,AGE,OCCUPATION_TYPE,GENDER,COUNT_PAID,COUNT_LATE,COUNT_MORE_THAN_90,COUNT_0_TO_60,AVG_DAYS,COUNT_MISSING_DAYS
0,A000614177,Housing Loan,2277986,91,1824507,22985.0,NaN,149.0,0,NaN,0000000000000000000000000000000000000000000000...,Joint,3780000.0,780,0,36,0,0.0,35,SEP,Male,17.0,0.0,0.0,0.0,0.0,1.0
1,A001744326,Business Loan,1030000,14,819209,30256.0,NaN,NaN,0,NaN,0000000000000000000000000000000000000000000000...,Joint,NaN,774,0,15,0,0.0,47,NaN,Male,18.0,0.0,0.0,0.0,0.0,0.0
2,A000242802,Housing Loan,960705,3,908604,25229.0,NaN,NaN,0,NaN,0000000000000000000000000000000000000000000000...,Joint,2622010.0,809,0,4,0,0.0,59,SENP,Female,18.0,0.0,0.0,0.0,0.0,0.0
3,B1000244547,Housing Loan,1437990,69,1367609,13067.0,NaN,259.0,0,NaN,0000000000000000000000000000000000000000000000...,Joint,2947100.0,759,0,36,0,0.0,54,SENP,Male,17.0,0.0,0.0,0.0,0.0,1.0
4,B3000178046,Business Loan,728474,1,718224,22350.0,21778.0,NaN,0,NaN,0000000000000000000000000000000000000000000000...,Joint,NaN,731,0,2,0,0.0,50,SENP,Male,18.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29978,A002307275,Housing Loan,949186,66,929479,8890.0,NaN,300.0,0,NaN,0000000000000000000000000000000000000000000000...,Joint,1190000.0,749,0,36,0,0.0,39,SALARIED,Male,17.0,0.0,0.0,0.0,0.0,1.0
29979,A000079323,Business Loan,765335,15,517748,28054.0,28054.0,36.0,0,NaN,000000000000000000000000000000000000XXX0000000...,Joint,NaN,743,0,16,0,0.0,43,NaN,Female,17.0,0.0,0.0,0.0,0.0,1.0
29980,A002385848,Housing Loan,920928,105,916673,9974.0,9974.0,374.0,0,NaN,0000000000000000000000000000000000000000000000...,Joint,1859200.0,745,0,36,0,0.0,46,SENP,Male,17.0,0.0,0.0,0.0,0.0,1.0
29981,A002435483,Housing Loan,1108527,0,1108527,NaN,NaN,NaN,21,2733.0,0000000000000000000000000000000000000000000000...,Joint,6226950.0,731,0,0,0,NaN,38,SALARIED,Female,17.0,1.0,0.0,1.0,21.0,0.0


In [15]:
df_new.to_csv('wrangled_data_main_test.csv', index=False)